# Trapalho Prático 2: Realidade Aumentada

### Nome: Pedro Geovanni Barbosa Ribeiro
### Matrícula: 2018054478

# Bibliotecas utilizadas

In [1]:
import cv2

import numpy as np

import matplotlib.pyplot as plt

from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *

from PIL import Image

from objloader import *

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


# Calibração da câmera

Resultados obtidos após a calibração no Octave

Calibration results after optimization (with uncertainties):

Focal Length:          fc = [ 409.92990   407.51958 ] +/- [ 15.80234   14.92167 ]

Principal point:       cc = [ 305.07598   223.87874 ] +/- [ 7.35134   13.69615 ]

Skew:             alpha_c = [ 0.00000 ] +/- [ 0.00000  ]   => angle of pixel axes = 90.00000 +
/- 0.00000 degrees

Distortion:            kc = [ 0.08372   -0.15473   -0.00711   -0.01103  0.00000 ] +/- [ 0.0759
4   0.61318   0.00826   0.00684  0.00000 ]

Pixel error:          err = [ 0.16554   0.21081 ]

# Detecção de Alvos

## Funções implementadas

#### Fun video_loader(num_frames = 0)

Essa função é responsável por carregar o vídeo que foi recebido junto com a documentação do trabalho e separá-lo em diferentes frames, para serem utilizados em etapas posteriores. Para realizar estas funções o método VideoCapture do opencv foi utilizado, em seguida utilizamos o método read() para pegar cada frame separadamente. Nessa função também se permite decidir quantos frames se deseja, para facilitar os testes executados durante o desenvolvimento do trabalho.

#### Fun image_pre_processing(images, bin_ts = 128)

Esta função é responsável por realizar o pré-processamento da imagem, para que ela possa futuramente passar pelo processo de detecção de bordas. Nesta função primeiros realizamos a conversão da imagem da escala RGB para a escala cinza atráves da função
cvtColor do opencv, em seguida realizamos o processo de binarização de imagem para facilitar a detecção de bordas, a binarização foi realizada atráves da função threshold do opencv. Nessa função podemos definir um limiar para a binarização, atráves de testes realizados dois valores de limiares foram satisfatórios 100 e 120, porém como apenas um limiar é permitido foi escolhido trabalhar com um limiar de 100. 

#### Fun edge_detection(image, low_ts = 100, max_ts = 200, show = False)

Aqui temos a função responsável por realizar a detecção de bordas, nela receberemos uma imagem já pré-processada pela função anterior e apenas executaremos a função Canny do opencv, para obter as bordas, para passarmos como parâmetro para função caany utilizamos os limiraes 100 e 200, que são os limiares padrões e funcionaram bem. Na função implementada também permitimos que, se desejado, possamos plotar as bordas detectadas.

#### Fun find_contours(edges)

Aqui nesta função nos detectamos os contornos das bordas já detectadas pela função anterior anteriormente, e nesta função executamos vários passos diferentes.

Primeiramente encontramos os contornos utilizando a função homônima da opencv findContours() que nos retornará uma lista de possíveis contornos na imagem. Em seguida nos utilizamos a função approxPolyDP para que posssamos coletar apenas os cotornos que são quadrados, outra filtragem que realizamos é evitar pegar contornos com área ou muito grandes ou muito pequenas, visando eliminar casos que não são desejados.

Tendo os quadrados e já eliminando as exceções, principalmente o maior quadrado da imagem, ordenamos os contornos restantes baseados em sua área, e armazenamos apenas os três com maior área. Por fim antes de retornarmos estes contornos temos que realizar um processamento para alterar seu formato, para que possam ser utilizados nas funções futuras.

Um ponto importante desta função é que não foi utilizada a função de detecção de quinas recomendada, cornerHarris(), já que atráves dos metódos realizados já foi possível fazer a detecção desejada.

#### Fun load_targets()

Está função é responsável por carregar a imagem do alvo e aplicar nela as mesmas transformações realizadas na função image_pre_processing(). Após o alvo já processado, o rotacionamos nos 4 possíveis ângulos(0°, 90°, 180° e 270°), e retornamos um array contendo o alvo em cada uma dessas rotações. 

#### Fun cross_correlation(image_1, image_2)

Aqui nós temos a função responsável por calcular a correlação cruzada normalizada entre duas imagens, para fórmula implementada foi utilizada como modelo a fórmula apresentada em sala de aula.

#### Fun homograph(image, contours, targets):

Aqui nos temos a função que irá realizar a homografia dos contornos encontrados no frame. Primeiramente armazenamos as quinas da imagem do alvo e executamos a função findHomography da opencv, utilizando como parâmetros o contorno encontrado e as quinas do alvo. Posteriormente realizamos a chamada da função warpPerspective para tranformar o contorno no mesmo formato que alvo. Por fim damos um resize na imagem obtida com a última etapa para que ela garantidamente fique com as mesmas dimensões que o alvo.

Agora com ambas as imagens podemos realizar o cálculo da correlação cruzada entre a imagem obtida e os alvos em diferentes
damos uma valor de 0 a 4, para avaliar a orientação da imagem, sendo de 0 a 3 representadando a orientação/90, e 4 representando que a imagem não é similar a nenhum dos alvos, apenas imagens que tiveram uma correlação cruzada menor do que 0.6 com o alvo em todas as rotoções recebem esse valor. Foi escolhida tal escala pois os índices de 0 a 3 serão utilizados na hora de indicar a orientação da imagem no funçaõ solvepnp().


In [2]:
def video_loader(num_frames=0):
    video = cv2.VideoCapture('entrada.mp4')
    images = []
    success, image = video.read()
    while success:
        images.append(image)
        success, image = video.read()
    
    if num_frames == 0:
        return images
    else:
        return images[:num_frames]

def image_pre_processing(images, bin_ts = 128):
    images_processed = []
    for image in images:
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        th, image_binary = cv2.threshold(image_gray, bin_ts, 255, cv2.THRESH_BINARY)
        images_processed.append(image_binary)
    
    return images_processed

def edge_detection(image, low_ts = 100, max_ts = 200, show = False):
    edges = cv2.Canny(image, low_ts, max_ts)
    
    if(show):
        plt.figure(figsize=(24,18))
        plt.imshow(edges, cmap='gray')
        plt.show()
        
    return edges

def find_contours(edges):
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    square_contours = []
    
    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)
        if len(approx) == 4 and cv2.contourArea(contour) > 400 and cv2.contourArea(contour) < 20000:
            square_contours.append(approx)
            
    square_contours_selected = sorted(square_contours, key = cv2.contourArea, reverse = True)[:3]
    
    formated_contours = []
    
    for element in square_contours_selected:
        formated_element = np.array([[element[0][0][0], element[0][0][1]],
                                     [element[1][0][0], element[1][0][1]],
                                     [element[2][0][0], element[2][0][1]],
                                     [element[3][0][0], element[3][0][1]]])
        formated_contours.append(formated_element)
        
    return formated_contours

def load_targets():
    target = cv2.imread('alvo.jpg')
    target = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)
    th, target = cv2.threshold(target, 100, 255, cv2.THRESH_BINARY)
    
    target90 = cv2.rotate(target, cv2.ROTATE_90_CLOCKWISE)
    target180 = cv2.rotate(target, cv2.ROTATE_180)
    target270 = cv2.rotate(target, cv2.ROTATE_90_COUNTERCLOCKWISE)
    
    return [target, target90, target180, target270]

def cross_correlation(image_1, image_2): 
    arr1 = np.subtract(image_1, np.mean(image_1))/np.std(image_1)
    arr2 = np.subtract(image_2, np.mean(image_2))/np.std(image_2)
    
    corr = (arr1 * arr2).sum() / image_1.size

    return corr

def homograph(image, contours, targets):
    target_points = np.array([[0,0], [0, targets[0].shape[1]-1], 
                              [targets[0].shape[0]-1, targets[0].shape[1]-1], [targets[0].shape[0]-1, 0]])
    
    homograph, mask = cv2.findHomography(contours, target_points, cv2.RANSAC)
    image_out = cv2.warpPerspective(image, homograph, (targets[0].shape[0], targets[0].shape[1]))
    
    image_out_gray = cv2.cvtColor(image_out, cv2.COLOR_BGR2GRAY)
    ret, image_out_binary = cv2.threshold(image_out_gray, 100, 55, cv2.THRESH_BINARY)
    
    resize_targets = [cv2.resize(targets[0], (image_out_binary.shape[1], image_out_binary.shape[0])), 
                      cv2.resize(targets[1], (image_out_binary.shape[1], image_out_binary.shape[0])), 
                      cv2.resize(targets[2], (image_out_binary.shape[1], image_out_binary.shape[0])), 
                      cv2.resize(targets[3], (image_out_binary.shape[1], image_out_binary.shape[0]))]
    
    
    correlation = 0
    orientation = 0
    
    for index in range(0, len(targets)):
        correlation_new = cross_correlation(image_out_binary, targets[index])
        if(correlation_new > correlation):
            correlation = correlation_new
            orientation = index
    
    if(correlation < 0.6):
        orientation = 4

    return orientation 

## Procedimento de detecção

Aqui nós executaremos as chamadas das funções implementadas anteriormente, os valores  das variáveis utilizadas apra as chamadas das funções são valores que foram decididos atráves de diferentes testes e os que foram escolhidos apresentaram o resultado mais satisfatório.

Foi decidido processar todos os 1100 frames do vídeo, para facilitar a execução das funções da opengl, já que já teremos boa parte dos dados necessários já calculados. Foi pensado em também realizar o pré-processamento do solvePnP, só que só foi notada essa possibilidade tarde demais.

In [3]:
images = video_loader()
targets = load_targets()
images_pre_processed = image_pre_processing(images, bin_ts=100)

list_contours = []
list_orientations = []
for frame in range(0, len(images)):
    edge = edge_detection(images_pre_processed[frame], low_ts = 100, max_ts = 200)
    contours = find_contours(edge)
    
    orientations = []
    for contour in contours:
        orientation = homograph(images[frame], contour, targets) 
        orientations.append(orientation)
    
    list_contours.append(contours)
    list_orientations.append(orientations)

C:\Users\danie\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in true_divide


# Exibição dos pikachus

#### Fun Pose(image_points, orientation)

Aqui é implementada a função de dectecção de pose, onde primeiramente carregamos os dados obtidos com a calibração
da câmera, matriz de parâmetros intrisecos e vetor de distorção, e criamos uma matriz que armazena o ponto de origem da imagem em cada rotação distinta. Posteriormente
executamos a função solvePnP da opencv para encontrar a pose da imagem, porém precisamos realizar a chamada da função 
Rodrigues da opencv para que possamos encontrar a matriz de rotação correta.

Com os valores obtidos até o momento podemos realizar a montagem da matriz de parâmetros extrinsecos, porém precisamos 
fazer a conversão da matriz das coordenadas do opencv para as coordenadas do opengl, poderiamos fazer isso multiplicando por uma matriz de conversão, mas preferi apenas alterar manualmente os sinais da segunda e terceira linhas da matriz de parâmetros extrinsecos.

Por fim precisamos transpor essa matriz antes de podermos retorná-la

In [4]:
def Pose(image_points, orientation):
    intrinsic_matrix = np.array([[409.92990, 0.0      , 305.07598],
                                 [0.0      , 407.51958, 223.87874],
                                 [0.0      , 0.0      , 1.0      ]])
    
    distortion = np.array([ 0.08372, -0.15473, -0.00711, -0.01103, 0.00000 ])
    
    object_points = np.array([[[-1.0, -1.0, 0.0], [ 1.0, -1.0, 0.0], [ 1.0,  1.0, 0.0], [-1.0,  1.0, 0.0]],
                              [[ 1.0, -1.0, 0.0], [ 1.0,  1.0, 0.0], [-1.0,  1.0, 0.0], [-1.0, -1.0, 0.0]],
                              [[ 1.0,  1.0, 0.0], [-1.0,  1.0, 0.0], [-1.0, -1.0, 0.0], [ 1.0, -1.0, 0.0]],
                              [[-1.0,  1.0, 0.0], [-1.0, -1.0, 0.0], [ 1.0, -1.0, 0.0], [ 1.0,  1.0, 0.0]]])
    
    success, rotation_vector, translation_vector = cv2.solvePnP(object_points[orientation], np.float32(image_points), 
                                            intrinsic_matrix, distortion, flags=cv2.SOLVEPNP_ITERATIVE)
    
    rotation_matrix = cv2.Rodrigues(rotation_vector)[0]
    
    extrinsic_matrix = np.array([[ rotation_matrix[0][0],  rotation_matrix[0][1],  rotation_matrix[0][2],  translation_vector[0]],
                                 [-rotation_matrix[1][0], -rotation_matrix[1][1], -rotation_matrix[1][2], -translation_vector[1]],
                                 [-rotation_matrix[2][0], -rotation_matrix[2][1], -rotation_matrix[2][2], -translation_vector[2]],
                                 [0.0                   , 0.0                   , 0.0                   ,  1.0                  ]])
    
    return extrinsic_matrix.T

#### Fun updateFrame()

Esta função é reponsável por atualizar o índice do frame que será utilizado na proxima cena, ela também incrementa o ângulo de rotação do pikachu, para faze-la ficar mais percepitivel o aumentamos de 3 em 3.

#### Fun drawBackground()

Esta função é responsável por pegar uma imagem e coloca-la como textura em um quadrado que ocupe a janela toda.

Primeiramente nos habilitamos o uso de texturas, já que será necessário e também desativamos a profundidade da janela, em seguida nos copiamos o frame que queremos utilizar como background, e realizamos um flip nela, já que caso contrário ela ficaria de cabeça para baixa na janela. Em seguida carregamos esta imagem como uma textura que será aplicada quando desenharmos o quadrado. Com a imagem como textura podemos desenhar o quadrado, primeiramente configuramos a matriz de projeção e de modelagem, para que ele fique correto, em seguida o desenhamos em si. por fim com o quadrado já com a textura desenhado habilitamos a profundidade da janela.

#### drawSquare()

Esta função é responsável por desenhar um quadrado onde o alvo foi detectado.

Primeiramente desabilitamos o uso de textura para que possamos desenhar o quadrado sem afetar nada, em seguida definimos uma grossura de linha que o faça ficar bem destacado quando na imagem, por teste foi escolhido uma grossura de 2.5, em seguida verificamos se os quadrados detectados são realmente alvos, se não forem (valor de orirentação igual a 4) iremos para o proximo quadrado detectado, caso tenhamos detectado um quadrado, nos utilizamos o glBegin(GL_LINE_LOOP) para desenhar um quadrado, a cor selecionada foi a mesma mostrada em sala, o verde. 

Um ponto interessante é que com invertemos a imagem, não podemos utilizar o Y encontrado, temos que fazer 480 - Y, para conseguirmos o valor de Y correto na hora de desenhar as linhas. Antes de terminarmos a função reativamos as texturas e retornamos o valor e a cor das linhas ao normal.

#### Fun initOpenGL(dimensions)

Esta função é responsável por construir a matriz de projeção correta, não foram feitas muitas alterações em relação a função que foi disponibilizada, somente adicionamos os parâmetros obtidos com a calibração de câmera, e as funções de fovy e aspect seguem as mesmas mostradas em sala de aula

#### Fun object3D(obj)

Esta função é responsável por desenhar todos os objetos que dependem do posicionamento da câmera, ou seja o cubo e o pikachu.

Primeiramente nós fazemos a checagem se a orientação não é 4, se for nos passamos para o próximo contorno, caso contrário nos iniciamos o processo de desenho das figuras. O primeiro passo é obter a posição da câmera, que é adquirida ao chamarmos a função Pose(image_points, orientation), passando como parâmetro o contorno e a orientação do mesmo. Ems seguida desabilitamos a profundidade e configuramos a posição da câmera, inicialmente alterando a matriz de projeão,em seguida carregando a perspectiva da câmera, e por fim antes de iniciarmos os desenhos, nós carrregamos a matriz de modelagem e a matriz contendo a pose da câmera.

Com tudo configurado podemos começar a desenhar as figuras, mas antes desabilitamos as texturas, depois utilizamos a função glutWireCube(2.0) para desenhar um cubo de tamanho 2x2x2, note que antes e depois de desenha-lo executamos um translate no cubo, isso serve para deixá-lo um pouco mais alinhado com o alvo em si. Em seguida ainda no cubo desenhamos uma linha que indica a orientação do cubo, está linha para ficar mais perceptivel é mais grossa que o cubo. Por fim desenhamos os pikachus na tela, so que antes disso habilitamos as texturas e profundidade, e aplicamos uma rotação ao pikachu. 

Com relação a rotação foi definido que caso o ponto inicial do contorno fosse acima de um limite em X e Y ele giraria em sentido horario, e caso contrário ele rodaria no sentido anti-horário, isto foi feito pois era de conhecido que apenas o alvo 2 deveria girar no sentido horário e ele sempre ficava na posição superior da janela.

#### Fun displayCallback()

Está função irá orientar o que iremos mostrar na tela.
 
Primeiramente nós carregamos a matriz de modelagem e limpamos o buffer de cor, assim limpando a imagem por completo. Em seguida chamamos a função drawBackground() para adicionar o frame atual como background, em seguida no chamamos a função drawSquare() para desenhar as localizações dos alvos, em seguida carregamos o modelo do pikachu e chamamos a função object3d(obj), para desenhar o cubo a orientação do cubo e o pikachu rotacionado.

Terminado tudo a ser desenhado damos um swap no buffer e chamamos a função updateFrame(), para atualizar o frame da proxima iteração do callback e aumentar o angulo de rotação.

#### Fun idleCallback()

Está é a função de idle, que não foi alterado em comparação com a fornecida

#### Main

Será o responsável por iniciar a janela do opengl, em comparação com o fornecido foi alterada apenas o nome da janela para "Realidade Aumentada - Pikachu"

In [5]:
frame = 0
rotation = 0

def updateFrame():
    global frame
    global rotation
    
    rotation += 3
    frame += 1
    
def drawBackground():
    global frame
    global images
    
    glEnable(GL_TEXTURE_2D)
    glDisable(GL_DEPTH_TEST)
    glDepthMask(GL_FALSE)
    
    image_background = images[frame].copy()
    image_background = cv2.cvtColor(image_background, cv2.COLOR_BGR2RGB)
    image_background = cv2.flip(image_background, 0)
    
    height, width, _ = image_background.shape
    
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, width, height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_background)
    glTexParameterf(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_NEAREST)
    
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluOrtho2D(0, width, 0, height)

    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()

    glBegin(GL_QUADS)
    glTexCoord2f(0.0, 0.0)
    glVertex2f(0.0, 0.0)
    glTexCoord2f(1.0, 0.0)
    glVertex2f(width, 0.0)
    glTexCoord2f(1.0, 1.0)
    glVertex2f(width, height)
    glTexCoord2f(0.0, 1.0)
    glVertex2f(0.0, height)
    glEnd()
    
    glEnable(GL_DEPTH_TEST)
    glDepthMask(GL_TRUE)
    
def drawSquare():
    global frame
    global list_contours
    global list_orientations
    
    glDisable(GL_TEXTURE_2D)
    glLineWidth(2.5)
    
    index = 0
    for contour in list_contours[frame]:
        if list_orientations[frame][index] == 4:
            continue
        glBegin(GL_LINE_LOOP);
        glColor3f(0.0, 1.0, 0.0);
        glVertex2f(contour[0][0], 480 - contour[0][1])
        glVertex2f(contour[1][0], 480 - contour[1][1])
        glVertex2f(contour[2][0], 480 - contour[2][1])
        glVertex2f(contour[3][0], 480 - contour[3][1])
        glEnd()
        index+=1
    
    glColor3fv((1,1,1))
    glLineWidth(1.0)
    glEnable(GL_TEXTURE_2D)
        
def initOpenGL(dimensions):
    (width, height) = dimensions
    
    glClearColor(0.0, 0.0, 0.0, 0.0)
    glClearDepth(1.0)

    glEnable(GL_DEPTH_TEST)

    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
 
    fovy = 2*np.arctan(0.5*480/407.51958)*180/np.pi;
    aspect = (640*407.51958)/(480*409.92990)
    gluPerspective(fovy, aspect, 0.1, 100.0)
        
def object3D(obj):
    global frame
    global rotation
    global list_contours
    global list_orientations
    
    for index in range(len(list_contours[frame])):
        if list_orientations[frame][index] == 4:
            continue
        
        pose = Pose(list_contours[frame][index], list_orientations[frame][index])
        
        glDisable(GL_DEPTH_TEST)
        
        glMatrixMode(GL_PROJECTION)
        glLoadIdentity()
        
        fovy = 2*np.arctan(0.5*480/407.51958)*180/np.pi;
        aspect = (640*407.51958)/(480*409.92990)
        gluPerspective(fovy, aspect, 0.1, 100.0)
        #gluOrtho2D(0, 640, 0, 480)

        glMatrixMode(GL_MODELVIEW)
        glLoadIdentity()
        
        glLoadMatrixf(pose)
        
        glDisable(GL_TEXTURE_2D)
        
        glTranslate(0, 0, 1)
        glutWireCube(2.0)
        glTranslate(0, 0, -1)
        
        glLineWidth(3)
        glBegin(GL_LINES);
        glVertex2f(0, 0)
        glVertex2f(-2, 0)
        glEnd()
        glLineWidth(1)
        
        glEnable(GL_TEXTURE_2D)
        glEnable(GL_DEPTH_TEST)
        
        if list_contours[frame][index][0][0] > 320 and 480 - list_contours[frame][index][0][1] > 240:
            glRotate(-rotation, 0,0,1)
        else:
            glRotate(rotation, 0,0,1)
        
        glCallList(obj.gl_list)
        
    
def displayCallback():
    glMatrixMode(GL_MODELVIEW)
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glLoadIdentity()
    
    drawBackground()
    
    drawSquare()
    
    obj = OBJ("Pikachu.obj", swapyz=True)
    
    object3D(obj) 
        
    glutSwapBuffers()
    updateFrame()
    

def idleCallback():
    glutPostRedisplay()
    
    
if __name__ == '__main__':
    dimensions = (640, 480)
    glutInit()
    glutInitDisplayMode(GLUT_RGBA | GLUT_DOUBLE)
    glutSetOption(GLUT_ACTION_ON_WINDOW_CLOSE, GLUT_ACTION_CONTINUE_EXECUTION)
    glutInitWindowSize(*dimensions)
    window = glutCreateWindow(b'Realidade Aumentada - Pikachu')
    
    initOpenGL(dimensions)
    
    glutDisplayFunc(displayCallback)
    glutIdleFunc(idleCallback)
    
    glutMainLoop()

C:\Users\danie\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


# Fontes utilizadas

a